# GEM Reconstruction with LC M001-related Transcriptomics — Enrichment Analysis of Subsystems

***by Kengo Watanabe***  

The maximum flux values, which were calculated using mouse genome-scale metabolic models (GEMs; Khodaee, S. et al. Sci. Rep. 2020) with the Longevity Consortium (LC) M001-related transcriptomics dataset (Tyshkovskiy, A. et al. Cell Metab. 2019), were compared.  
–> In this notebook, enrichment analysis is performed for the GEM subsystems in the significantly flux-changed reactions. This notebook is written by R.  

Input:  
* Cleaned reaction metadata: 220610_LC-M001-related-transcriptomics-GEM-ver2-4_FluxAnalysis_reaction-metadata.xlsx  
* Cleaned flux data: 220610_LC-M001-related-transcriptomics-GEM-ver2-4_FluxAnalysis_max-flux-raw-data.tsv  
* Flux comparison summary: 220610_LC-M001-related-transcriptomics-GEM-ver2-4_StatisticalTest_flux-comparison.xlsx  

Output:  
* Figure 4e  
* Supplementary Figure 2a, b, g  
* Supplementary Data 4  

Original notebook (memo for my future tracing):  
* dalek:[JupyterLab HOME]/220606_LC-M001-related-transcriptomics-GEM/220616_LC-M001-related-transcriptomics-GEM-ver2-4_Enrichment2.ipynb  

In [ ]:
library("tidyverse")
options(repr.plot.width=5, repr.plot.height=5)#Default=7x7

#Bioconductor
for (package in c("clusterProfiler", "enrichplot")) {
    #if (!requireNamespace("BiocManager", quietly=TRUE))
    #    install.packages("BiocManager")
    #BiocManager::install(package)
    eval(bquote(library(.(package))))
    print(str_c(package, ": ", as.character(packageVersion(package))))
}
#CRAN
for (package in c("readxl", "openxlsx")) {
    #install.packages(package)
    eval(bquote(library(.(package))))
    print(str_c(package, ": ", as.character(packageVersion(package))))
}

## 1. All the successfully flux-calculated reactions

In [ ]:
#Import reaction metadata
fileDir <- "./ExportData/"
ipynbName <- "220610_LC-M001-related-transcriptomics-GEM-ver2-4_FluxAnalysis_"
fileName <- "reaction-metadata.xlsx"
sheetName <- "Reaction"
temp <- read_excel(str_c(fileDir,ipynbName,fileName), sheet=sheetName)

print(str_c("nrow: ",nrow(temp)))
head(temp)
print(str_c("Unique reaction: ", length(unique(temp$ReactionID))))
print(str_c("Unique subsystem: ", length(unique(temp$Subsystem))))

meta_tbl <- temp

In [ ]:
#Check
meta_tbl %>% dplyr::filter(is.na(Subsystem))

> –> Reactions without subsytem annotation should be eliminated in this enrichment analysis. Of note, Priyanka confirmed they were not errors.  

In [ ]:
#Eliminate reactions without subsystem
temp <- meta_tbl %>%
    tidyr::drop_na(Subsystem)

print(str_c("nrow: ",nrow(temp)))
print(str_c("Unique reaction: ", length(unique(temp$ReactionID))))
print(str_c("Unique subsystem: ", length(unique(temp$Subsystem))))

bgd_tbl <- temp

In [ ]:
#Import flux data
fileDir <- "./ExportData/"
ipynbName <- "220610_LC-M001-related-transcriptomics-GEM-ver2-4_FluxAnalysis_"
fileName <- "max-flux-raw-data.tsv"
temp <- read_delim(str_c(fileDir,ipynbName,fileName), delim="\t")
print(str_c("nrow: ",nrow(temp)))

#Take the successfully calculated reactions
temp <- bgd_tbl %>%
    dplyr::filter(ReactionID %in% temp$ReactionID)

print(str_c("nrow: ",nrow(temp)))
head(temp)
print(str_c("Unique reaction: ", length(unique(temp$ReactionID))))
print(str_c("Unique subsystem: ", length(unique(temp$Subsystem))))

bgd_tbl <- temp

## 2. Significantly changed reactions

In [ ]:
#Import the summary tables
fileDir <- "./ExportData/"
ipynbName <- "220610_LC-M001-related-transcriptomics-GEM-ver2-4_StatisticalTest_"
fileName <- "flux-comparison.xlsx"
sheetName <- "Main_vs-each-control"
temp <- read_excel(str_c(fileDir,ipynbName,fileName), sheet=sheetName)
print(str_c("Original nrow: ",nrow(temp)))

#Clean
temp1 <- str_subset(names(temp), "_AdjPval")
temp <- temp %>%
    dplyr::select(ReactionID, all_of(temp1)) %>%
    tidyr::drop_na(Intervention_AdjPval)#NA = reactions having invariable flux values
names(temp) <- str_replace(names(temp), "_AdjPval", "")

print(str_c("nrow: ",nrow(temp)))
head(temp)
print(str_c("Unique reaction: ", length(unique(temp$ReactionID))))

pval_tbl1 <- temp

In [ ]:
#Import the summary tables
fileDir <- "./ExportData/"
ipynbName <- "220610_LC-M001-related-transcriptomics-GEM-ver2-4_StatisticalTest_"
fileName <- "flux-comparison.xlsx"
sheetName <- "Posthoc2_MR-vs-CR"
temp <- read_excel(str_c(fileDir,ipynbName,fileName), sheet=sheetName)
print(str_c("Original nrow: ",nrow(temp)))

#Clean
temp1 <- str_subset(names(temp), "_AdjPval")
temp <- temp %>%
    dplyr::select(ReactionID, all_of(temp1)) %>%
    dplyr::filter(ReactionID %in% pval_tbl1$ReactionID)
names(temp) <- str_replace(names(temp), "_AdjPval", "")

print(str_c("nrow: ",nrow(temp)))
head(temp)
print(str_c("Unique reaction: ", length(unique(temp$ReactionID))))

pval_tbl2 <- temp

## 3. Enrichment Analysis

### 3-1. Aca, MR, GHRKO, SnellDW vs. each control

> According to the number of significantly changed reactions, the four interventions (Aca, MR, GHRKO, SnellDW) are analyzed.  

#### 3-1-1. Save result objects

In [ ]:
comparison_vec <- c("Acar-vs-Con1", "MetR-vs-Con2", "GHRk-vs-GHRw", "SneD-vs-SneW")

#Create a workbook object to save as one single .xlsx file
workbook <- createWorkbook()

#Summarize results per comparison
nRxns_vec <- c()
nMappedRxns_vec <- c()
nRxnSystems_vec <- c()
res_list <- list()
for (i in 1:length(comparison_vec)) {
    #Prepare input for enricher()
    comparison <- comparison_vec[i]
    rxns <- pval_tbl1 %>%
        dplyr::filter(Intervention<0.05) %>%
        dplyr::filter(!!as.name(comparison)<0.05) %>%
        .$ReactionID
    print(str_c(comparison,": ",length(rxns)," changed reactions"))
    bgds <- bgd_tbl %>%
        dplyr::select(Subsystem, ReactionID)
    labels <- bgd_tbl %>%
        dplyr::mutate(Label=Subsystem) %>%#Dummy in this case
        dplyr::select(Subsystem, Label) %>%
        dplyr::distinct()
    
    #Save info
    nRxns_vec <- c(nRxns_vec, length(rxns))
    temp <- bgds %>%
        dplyr::filter(ReactionID %in% rxns)
    nMappedRxns_vec <- c(nMappedRxns_vec, length(unique(temp$ReactionID)))
    nRxnSystems_vec <- c(nRxnSystems_vec, length(unique(temp$Subsystem)))
    
    #Enrichment analysis
    temp <- enricher(gene=rxns,
                     pvalueCutoff=1.0,#To export all
                     pAdjustMethod="BH",
                     #universe=backgrounds,#Already managed
                     minGSSize=4,
                     maxGSSize=10000,
                     qvalueCutoff=1.0,#To export all
                     TERM2GENE=bgds,
                     TERM2NAME=labels)
    
    #Add the summary table to the workbook object as an independent sheet
    if (is.data.frame(temp[])) {
        temp1 <- tibble(temp[]) %>%
            dplyr::select(-Description) %>%
            dplyr::rename(Subsystem=ID, Ratio2ChangedRxns=GeneRatio, Ratio2BGs=BgRatio,
                          Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedChangedRxn=geneID, nMappedChangedRxns=Count)
    } else {
        temp1 <- tibble(`n/a`=NA)
    }
    addWorksheet(workbook, sheetName=comparison)
    writeData(workbook, comparison, temp1)
    
    #Add result object to list
    res_list <- c(res_list, list(temp))
}

#Save the workbook as one single .xlsx file
fileDir <- "./ExportData/"
ipynbName <- "220616_LC-M001-related-transcriptomics-GEM-ver2-4_Enrichment2_"
fileName <- "clusterProfiler-results.xlsx"
saveWorkbook(workbook, file=str_c(fileDir,ipynbName,fileName), overwrite=TRUE)

print(str_c("nObjects: ", as.character(length(res_list))))

#### 3-1-2. Aca

In [ ]:
obj_i <- 1
figtitle <- "Acarbose vs. Control 1"

#Retreive results
comparison <- comparison_vec[obj_i]
nRxns <- nRxns_vec[obj_i]
nMappedRxns <- nMappedRxns_vec[obj_i]
nRxnSystems <- nRxnSystems_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(comparison)
print(str_c(" - # of the changed reactions: ",as.character(nRxns)))
print(str_c(" - # of the changed reactions that were mapped to any subsystem: ",as.character(nMappedRxns)))
print(str_c(" - # of subsystems having any changed reactions as a member: ",as.character(nRxnSystems)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::select(-Description) %>%
        dplyr::rename(Subsystem=ID, Ratio2ChangedRxns=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedChangedRxn=geneID, nMappedChangedRxns=Count)
}

#Visualization
display <- tibble(res[]) %>%
    dplyr::filter(pvalue<0.05) %>%
    nrow()
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=10, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 275), breaks=seq(0, 250, by=50), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=50)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of the changed reactions",
             y="", title=str_c("Enriched subsystems: ",figtitle)) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica"),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=8.25, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "220616_LC-M001-related-transcriptomics-GEM-ver2-4_Enrichment2_"
    fileName <- str_c(comparison,".pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=8.25, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-1-3. MR

In [ ]:
obj_i <- 2
figtitle <- "MR diet vs. Control 2"

#Retreive results
comparison <- comparison_vec[obj_i]
nRxns <- nRxns_vec[obj_i]
nMappedRxns <- nMappedRxns_vec[obj_i]
nRxnSystems <- nRxnSystems_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(comparison)
print(str_c(" - # of the changed reactions: ",as.character(nRxns)))
print(str_c(" - # of the changed reactions that were mapped to any subsystem: ",as.character(nMappedRxns)))
print(str_c(" - # of subsystems having any changed reactions as a member: ",as.character(nRxnSystems)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::select(-Description) %>%
        dplyr::rename(Subsystem=ID, Ratio2ChangedRxns=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedChangedRxn=geneID, nMappedChangedRxns=Count)
}

#Visualization
display <- tibble(res[]) %>%
    dplyr::filter(pvalue<0.05) %>%
    nrow()
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=10, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 275), breaks=seq(0, 250, by=50), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=50)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of the changed reactions",
             y="", title=str_c("Enriched subsystems: ",figtitle)) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica"),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=8.25, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "220616_LC-M001-related-transcriptomics-GEM-ver2-4_Enrichment2_"
    fileName <- str_c(comparison,".pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=8.25, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-1-3. GHRKO

In [ ]:
obj_i <- 3
figtitle <- "GHR knockout vs. wild-type"

#Retreive results
comparison <- comparison_vec[obj_i]
nRxns <- nRxns_vec[obj_i]
nMappedRxns <- nMappedRxns_vec[obj_i]
nRxnSystems <- nRxnSystems_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(comparison)
print(str_c(" - # of the changed reactions: ",as.character(nRxns)))
print(str_c(" - # of the changed reactions that were mapped to any subsystem: ",as.character(nMappedRxns)))
print(str_c(" - # of subsystems having any changed reactions as a member: ",as.character(nRxnSystems)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::select(-Description) %>%
        dplyr::rename(Subsystem=ID, Ratio2ChangedRxns=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedChangedRxn=geneID, nMappedChangedRxns=Count)
}

#Visualization
display <- tibble(res[]) %>%
    dplyr::filter(pvalue<0.05) %>%
    nrow()
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=10, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 275), breaks=seq(0, 250, by=50), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=50)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of the changed reactions",
             y="", title=str_c("Enriched subsystems: ",figtitle)) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica"),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=8.25, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "220616_LC-M001-related-transcriptomics-GEM-ver2-4_Enrichment2_"
    fileName <- str_c(comparison,".pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=8.25, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

#### 3-1-4. SnellDW

In [ ]:
obj_i <- 4
figtitle <- "Snell dwarf vs. wild-type"

#Retreive results
comparison <- comparison_vec[obj_i]
nRxns <- nRxns_vec[obj_i]
nMappedRxns <- nMappedRxns_vec[obj_i]
nRxnSystems <- nRxnSystems_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(comparison)
print(str_c(" - # of the changed reactions: ",as.character(nRxns)))
print(str_c(" - # of the changed reactions that were mapped to any subsystem: ",as.character(nMappedRxns)))
print(str_c(" - # of subsystems having any changed reactions as a member: ",as.character(nRxnSystems)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::select(-Description) %>%
        dplyr::rename(Subsystem=ID, Ratio2ChangedRxns=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedChangedRxn=geneID, nMappedChangedRxns=Count)
}

#Visualization
display <- tibble(res[]) %>%
    dplyr::filter(pvalue<0.05) %>%
    nrow()
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=10, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 275), breaks=seq(0, 250, by=50), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=50)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of the changed reactions",
             y="", title=str_c("Enriched subsystems: ",figtitle)) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica"),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=8.25, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "220616_LC-M001-related-transcriptomics-GEM-ver2-4_Enrichment2_"
    fileName <- str_c(comparison,".pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=8.25, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

### 3-2. MR vs. CR

#### 3-2-1. Save result objects

> Add the result to the above file

In [ ]:
comparison_vec <- c("MetR-vs-CalR")

#Load the existing .xlsx file as a new workbook object
fileDir <- "./ExportData/"
ipynbName <- "220616_LC-M001-related-transcriptomics-GEM-ver2-4_Enrichment2_"
fileName <- "clusterProfiler-results.xlsx"
workbook <- loadWorkbook(str_c(fileDir,ipynbName,fileName))

#Summarize results per comparison
nRxns_vec <- c()
nMappedRxns_vec <- c()
nRxnSystems_vec <- c()
res_list <- list()
for (i in 1:length(comparison_vec)) {
    #Prepare input for enricher()
    comparison <- comparison_vec[i]
    temp <- pval_tbl1 %>%
        dplyr::filter(Intervention<0.05)
    rxns <- pval_tbl2 %>%
        dplyr::filter(ReactionID %in% temp$ReactionID) %>%
        dplyr::filter(!!as.name(comparison)<0.05) %>%
        .$ReactionID
    print(str_c(comparison,": ",length(rxns)," changed reactions"))
    bgds <- bgd_tbl %>%
        dplyr::select(Subsystem, ReactionID)
    labels <- bgd_tbl %>%
        dplyr::mutate(Label=Subsystem) %>%#Dummy in this case
        dplyr::select(Subsystem, Label) %>%
        dplyr::distinct()
    
    #Save info
    nRxns_vec <- c(nRxns_vec, length(rxns))
    temp <- bgds %>%
        dplyr::filter(ReactionID %in% rxns)
    nMappedRxns_vec <- c(nMappedRxns_vec, length(unique(temp$ReactionID)))
    nRxnSystems_vec <- c(nRxnSystems_vec, length(unique(temp$Subsystem)))
    
    #Enrichment analysis
    temp <- enricher(gene=rxns,
                     pvalueCutoff=1.0,#To export all
                     pAdjustMethod="BH",
                     #universe=backgrounds,#Already managed
                     minGSSize=4,
                     maxGSSize=10000,
                     qvalueCutoff=1.0,#To export all
                     TERM2GENE=bgds,
                     TERM2NAME=labels)
    
    #Add the summary table to the workbook object as an independent sheet
    if (is.data.frame(temp[])) {
        temp1 <- tibble(temp[]) %>%
            dplyr::select(-Description) %>%
            dplyr::rename(Subsystem=ID, Ratio2ChangedRxns=GeneRatio, Ratio2BGs=BgRatio,
                          Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedChangedRxn=geneID, nMappedChangedRxns=Count)
    } else {
        temp1 <- tibble(`n/a`=NA)
    }
    addWorksheet(workbook, sheetName=comparison)
    writeData(workbook, comparison, temp1)
    
    #Add result object to list
    res_list <- c(res_list, list(temp))
}

#Update the existing .xlsx file
saveWorkbook(workbook, file=str_c(fileDir,ipynbName,fileName), overwrite=TRUE)

print(str_c("nObjects: ", as.character(length(res_list))))

#### 3-2-2. MR vs. CR

In [ ]:
obj_i <- 1
figtitle <- "MR diet vs. CR diet"

#Retreive results
comparison <- comparison_vec[obj_i]
nRxns <- nRxns_vec[obj_i]
nMappedRxns <- nMappedRxns_vec[obj_i]
nRxnSystems <- nRxnSystems_vec[obj_i]
res <- res_list[[obj_i]]

#Check
print(comparison)
print(str_c(" - # of the changed reactions: ",as.character(nRxns)))
print(str_c(" - # of the changed reactions that were mapped to any subsystem: ",as.character(nMappedRxns)))
print(str_c(" - # of subsystems having any changed reactions as a member: ",as.character(nRxnSystems)))
res
print(" <- Note that the above 'X enriched terms found' is not correct. In this case, X indicates the number of all the tested terms.")
if (is.data.frame(res[])) {
    tibble(res[]) %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::select(-Description) %>%
        dplyr::rename(Subsystem=ID, Ratio2ChangedRxns=GeneRatio, Ratio2BGs=BgRatio,
                      Pval=pvalue, AdjPval=p.adjust, Qval=qvalue, MappedChangedRxn=geneID, nMappedChangedRxns=Count)
}

#Visualization
display <- tibble(res[]) %>%
    dplyr::filter(pvalue<0.05) %>%
    nrow()
if (display>0) {
    temp <- res %>%
        dplyr::filter(pvalue<0.05) %>%#Display only nominal P-value < 0.05
        dplyr::mutate(PvalLabel=str_c("AdjPval = ",scales::scientific(p.adjust, digits=2)),
                      AdjSignif=ifelse(p.adjust<0.05, "True", "False")) %>%
        barplot(., x="Count", color="p.adjust", showCategory=display) +
        geom_text(aes(label=PvalLabel, color=AdjSignif), nudge_x=10, hjust=0) +
        coord_cartesian(clip="off") +
        scale_x_continuous(limits=c(0, 275), breaks=seq(0, 250, by=50), expand=c(0, 0)) +
        scale_y_discrete(labels=function(x) {str_wrap(x, width=50)}) +
        scale_fill_viridis_c(begin=0, end=1, direction=1, option="plasma",
                             limits=c(0, 0.2), breaks=seq(0, 0.2, by=0.05), name="AdjPval") +
        scale_color_manual(values=c("True"="#990000", "False"="gray40"), ) +
        guides(fill=guide_colorbar(reverse=TRUE), color="none") +
        labs(x="Count of the changed reactions",
             y="", title=str_c("Enriched subsystems: ",figtitle)) +
        theme_classic(base_size=16, base_family="Helvetica") +
        theme(text=element_text(face="plain", color="black", family="Helvetica")) +
        theme(axis.text.x=element_text(face="plain", color="black", family="Helvetica"),
              axis.text.y=element_text(face="plain", color="black", family="Helvetica"),
              axis.title=element_text(face="plain", color="black", family="Helvetica")) +
        theme(plot.title=element_text(size=18, hjust=1.0)) +
        theme(legend.direction="vertical", legend.box="horizontal",
              legend.background=element_blank())
    options(repr.plot.width=8.25, repr.plot.height=max(c(1+display*0.25, 2.5)))
    plot(temp)
    #Save
    fileDir <- "./ExportFigures/"
    ipynbName <- "220616_LC-M001-related-transcriptomics-GEM-ver2-4_Enrichment2_"
    fileName <- str_c(comparison,".pdf")
    ggsave(file=str_c(fileDir,ipynbName,fileName), plot=temp,
           width=8.25, height=max(c(1+display*0.25, 2.5)), units="in")
    #(Font family is not reflected in JupyterLab output, but correctly done in .pdf file.)
}

# — Session information —

In [ ]:
sessionInfo()